In [2]:
from bs4 import BeautifulSoup  # For parsing HTML
import requests  # For making HTTP requests
import pandas as pd
import numpy as np

##### Scrap Individual Player Stats

In [5]:
#Get top 100 clubs
url_club = 'https://www.transfermarkt.com/spieler-statistik/wertvollstemannschaften/marktwertetop'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response_club= requests.get(url_club, headers=headers)
soup_club = BeautifulSoup(response_club.content, 'html.parser')

In [13]:
url = 'https://www.transfermarkt.com/manchester-united/kader/verein/985/plus/1/galerie/0?saison_id=2022'
# url = 'https://www.transfermarkt.com/david-de-gea/profil/spieler/59377'
# page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [39]:
links = soup.find_all("tr")

In [83]:
data_table = soup.find_all('table', class_=['items'])

In [104]:
column_headers = []
player_detail = {}
for head in data_table[0].find_all('thead'):
    for a in head.find_all('th'):
        column_headers.append(a.text.strip())

count = 0
for tr in data_table[0].find_all('tr'):
    player_detail[count] = []
    for td in tr.find_all('td', class_=['zentriert', 'hauptlink']):
        if td.text.strip():
            player_detail[count].append(td.text.strip())
        for img in td.find_all('img'):
            player_detail[count].append(img['alt'].strip())
    count += 1

keys_to_remove = []
for key, value in player_detail.items():
    if len(value) < 5:
        keys_to_remove.append(key)

for key in keys_to_remove:
    del player_detail[key]

# player_detail


['#', 'Player', 'Date of birth / Age', 'Nat.', 'Height', 'Foot', 'Joined', 'Signed from', 'Contract', 'Market value']


{1: ['-',
  'Dean Henderson',
  'Mar 12, 1997 (26)',
  'England',
  '1,88m',
  'right',
  'Aug 1, 2020',
  'Manchester United U23',
  'Jun 30, 2025',
  '€22.00m'],
 4: ['1',
  'David de Gea',
  'Nov 7, 1990 (32)',
  'Spain',
  '1,89m',
  'right',
  'Jul 1, 2011',
  'Atlético de Madrid',
  'Jun 30, 2023',
  '€15.00m'],
 7: ['31',
  'Jack Butland',
  'Mar 10, 1993 (30)',
  'England',
  '1,96m',
  'right',
  'Jan 6, 2023',
  'Crystal Palace',
  'Jun 30, 2023',
  '€1.50m'],
 10: ['22',
  'Tom Heaton',
  'Apr 15, 1986 (37)',
  'England',
  '1,88m',
  'right',
  'Jul 2, 2021',
  'Aston Villa',
  'Jun 30, 2023',
  '€600k'],
 13: ['6',
  'Lisandro Martínez',
  'Jan 18, 1998 (25)',
  'Argentina',
  '1,75m',
  'left',
  'Jul 27, 2022',
  'Ajax Amsterdam',
  'Jun 30, 2027',
  '€50.00m'],
 16: ['19',
  'Raphaël Varane',
  'Apr 25, 1993 (30)',
  'France',
  'Martinique',
  '1,91m',
  'right',
  'Aug 14, 2021',
  'Real Madrid',
  'Jun 30, 2025',
  '€40.00m'],
 19: ['5',
  'Harry Maguire',
  'Mar 5, 

In [112]:
df = pd.DataFrame(columns=column_headers)

for idx, data in player_detail.items():
    if len(data) == 10:    df.loc[len(df)] = data
    elif len(data) == 11: 
        data.pop(4)
        df.loc[len(df)] = data   
df

,#,Player,Date of birth / Age,Nat.,Height,Foot,Joined,Signed from,Contract,Market value
0,-,Dean Henderson,"Mar 12, 1997 (26)",England,"1,88m",right,"Aug 1, 2020",Manchester United U23,"Jun 30, 2025",€22.00m
1,1,David de Gea,"Nov 7, 1990 (32)",Spain,"1,89m",right,"Jul 1, 2011",Atlético de Madrid,"Jun 30, 2023",€15.00m
2,31,Jack Butland,"Mar 10, 1993 (30)",England,"1,96m",right,"Jan 6, 2023",Crystal Palace,"Jun 30, 2023",€1.50m
3,22,Tom Heaton,"Apr 15, 1986 (37)",England,"1,88m",right,"Jul 2, 2021",Aston Villa,"Jun 30, 2023",€600k
4,6,Lisandro Martínez,"Jan 18, 1998 (25)",Argentina,"1,75m",left,"Jul 27, 2022",Ajax Amsterdam,"Jun 30, 2027",€50.00m
5,19,Raphaël Varane,"Apr 25, 1993 (30)",France,"1,91m",right,"Aug 14, 2021",Real Madrid,"Jun 30, 2025",€40.00m
6,5,Harry Maguire,"Mar 5, 1993 (30)",England,"1,94m",right,"Aug 5, 2019",Leicester City,"Jun 30, 2025",€25.00m
7,2,Victor Lindelöf,"Jul 17, 1994 (28)",Sweden,"1,87m",right,"Jul 1, 2017",SL Benfica,"Jun 30, 2024",€15.00m
8,-,Axel Tuanzebe,"Nov 14, 1997 (25)",England,"1,85m",right,"Jan 1, 2017",Manchester United U18,"Jun 30, 2023",€4.50m
9,4,Phil Jones,"Feb 21, 1992 (31)",England,"1,85m",right,"Jul 1, 2011",Blackburn Rovers,"Jun 30, 2023",€2.00m


##### Scrap Player Contract